In [1]:
import os
import sys
import glob

from subprocess import Popen, PIPE, STDOUT, call

In [55]:
from set_test import list_test

In [3]:
files = sorted(glob.glob('./tasks/*'))
files

['./tasks/__pycache__',
 './tasks/task1.py',
 './tasks/task2.py',
 './tasks/task3.py']

In [4]:
os.system('python ./tasks/task1.py')

256

In [5]:
py = 'python'
file = './tasks/task1.py'

p = Popen('python ./tasks/task1.py',
          stdout=PIPE, stdin=PIPE, stderr=STDOUT, shell=True)
grep_stdout = p.communicate(input=b'3\n3\n3\n', timeout=6*60)[0]
#type(grep_stdout.decode('866', errors='ignore')) # Для винды 10 кодировка по умолчанию 866
grep_stdout.decode()

'9\n'

In [5]:
#grep_stdout.decode('866', errors='ignore')
grep_stdout.decode()

'9\n'

In [34]:
p = Popen('python -m pycodestyle --ignore=E265,W292 ./tasks/task1.py', stdout=PIPE, stdin=PIPE, stderr=STDOUT, shell=True)
grep_stdout = p.communicate()[0]
#type(grep_stdout.decode('866', errors='ignore')) # Для винды 10 кодировка по умолчанию 866
grep_stdout.decode()

''

In [35]:
def pep8_checker(filename):
    """
    Check codestyle by pycodestyle
    """
    p = Popen(f'python -m pycodestyle --ignore=E265,W292 {filename}',
              stdout=PIPE, stdin=PIPE, stderr=STDOUT, shell=True)
    grep_stdout = p.communicate()[0]
    #type(grep_stdout.decode('866', errors='ignore')) # Для винды 10 кодировка по умолчанию 866
    pystyle_message = grep_stdout.decode()
    if pystyle_message:
        return 'NOPEP8', pystyle_message
    else:
        return 'OK', 'Your codestyle is PEP8'

In [36]:
pep8_checker('./tasks/task1.py')

('OK', 'Your codestyle is PEP8')

In [25]:
def result_checker(filename, input_data, output_data):
    """
    Check results of programs
    """
    p = Popen(f'python {filename}', stdout=PIPE, stdin=PIPE, stderr=STDOUT, shell=True)
    grep_stdout = p.communicate(input=input_data, timeout=6*60)[0]
    if grep_stdout.decode() == output_data:
        return "Test is passed"
    else:
        return "Test isn't passed", grep_stdout.decode()

In [33]:
test_input = [b'3\n3\n3\n', b'1\n1\n-2\n', b'3\n3\n3\n', b'3\n3\n3\n', b'3\n3\n3\n']
test_output = ['9\n', '0\n', '9\n','9\n','9\n']

for tst_in, tst_out in zip(test_input, test_output):
    print(result_checker('./tasks/task1.py', tst_in, tst_out))

Test is passed
Test is passed
Test is passed
Test is passed
Test is passed


## Создем код, способный проверять функции в модулях

In [52]:
import importlib.util
import multiprocessing, time, signal

In [195]:
default_path = (
    f'/home/andr/Yandex.Disk/3.Programing/'
    f'balovstvo&checking/test_interesting_code/checker_task'
)
task_path = './tasks/'
os.chdir(default_path)

In [196]:
def sleeper():
    time.sleep(10)
    
def executor(file, name_function, arguments):
    try:
        exec(f"global result; result = spec.{name_function}{arguments}")
        return_dict['result'] = result
    except Exception as e:
        return_dict['result'] = (f'Some errors in file {file}:'
                                 f' {sys.exc_info()[0]}: {sys.exc_info()[1]} ')
    return None

In [197]:
def result_checker_function(filename, name_function, input_data, output_data):
    sl = multiprocessing.Process(target=sleeper)
    wr = multiprocessing.Process(target=executor, args=(filename, name_function, input_data))
    
    sl.start()
    wr.start()
    
    while True:
        if not sl.is_alive():
            wr.terminate()
            return_dict['result'] = f'TimeoutError. Endless loop is detected in {filename}.'
            break
        elif not wr.is_alive():
            sl.terminate()
            break 

    if return_dict['result'] == output_data:
        return "Test is passed", "It is correct."
    else:
        return "Test isn't passed", return_dict['result']
    del spec
    
    return None

In [203]:
os.chdir(default_path)
for test in list_test:
    if test[0] == 'codeinout':
        print('Run check input and output')
    elif test[0] == 'function':
        for idx_test, item_test in enumerate(test[-1]):
            os.chdir(task_path)
            manager = multiprocessing.Manager()
            return_dict = manager.dict()
            try:
                spec = importlib.import_module('task2', 'task2.py')
                importlib.reload(spec)
                msg, out = result_checker_function(
                    'task2.py', 
                    test[1], 
                    item_test[0], 
                    item_test[1]
                )
            except Exception as e:
                msg = "Test isn't passed"
                out = (f'Some errors in file task2.py:'
                                 f' {sys.exc_info()[0]}: {sys.exc_info()[1]} ')
                os.chdir(default_path)
                
            if msg == "Test is passed":
                print("Test is passed")
            elif msg == "Test isn't passed":
                print(item_test[0], item_test[1], out)
            else:
                print('Unexpected condition!!!')
            
            try:
                del sys.modules['task2']
            except KeyError:
                pass
            
            os.chdir(default_path)

(3, 6, 2) [3, 5] TimeoutError. Endless loop is detected in task2.py.
(0, 10, 3) [0, 3, 6, 9] TimeoutError. Endless loop is detected in task2.py.
(10, 0, 3) [] TimeoutError. Endless loop is detected in task2.py.
(10, 0, -3) [10, 7, 4, 1] TimeoutError. Endless loop is detected in task2.py.
(-5, 10, 3) [-5, -2, 1, 4, 7] TimeoutError. Endless loop is detected in task2.py.
Run check input and output


In [192]:
del sys.modules['task2']

KeyError: 'task2'

In [191]:
spec

NameError: name 'spec' is not defined

Sleeper stoped
Process worker was stopped


In [37]:
sl.is_alive()

False

In [38]:
wr.is_alive()

False